<a href="https://colab.research.google.com/github/DianaTixi/CNN_FP_DT/blob/main/GeneracionTexto_Ejemplo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Librerias
%%time
%%capture
!pip install transformers

CPU times: user 97.2 ms, sys: 26.7 ms, total: 124 ms
Wall time: 11.6 s


In [2]:
!nvidia-smi

Mon May  9 19:25:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pip install --upgrade pandas==1.2.5

     |████████████████████████████████| 9.9 MB 5.3 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [3]:
!pip3 install pickle5
import pickle5 as pickle
with open('articles.pkl', "rb") as fh:
  data = pickle.load(fh)

In [4]:
import re
import json
from sklearn.model_selection import train_test_split


with open('recipes.json') as f:
    data = json.load(f)

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts['Instructions']).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

train, test = train_test_split(data,test_size=0.15) 


build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 10361
Test dataset length: 1829


#### El siguiente paso es descargar el tokenizador, que usamos. Usamos el tokenizador del modelo german-gpt2 en huggingface.

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("anonymous-german-nlp/german-gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/948k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


###Inicialice Trainer con TrainingArguments y el modelo GPT-2

La clase Trainer proporciona una API para el entrenamiento completo de características. Se utiliza en la mayoría de los scripts de ejemplo de Huggingface. Antes de que podamos crear una instancia de nuestro Entrenador, debemos descargar nuestro modelo GPT-2 y crear un TrainingArguments para acceder a todos los puntos de personalización durante el entrenamiento. En TrainingArguments, podemos definir los hiperparámetros que vamos a utilizar en el proceso de entrenamiento, como nuestro learning_rate, num_train_epochs o per_device_train_batch_size.

In [7]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("anonymous-german-nlp/german-gpt2")


training_args = TrainingArguments(
    output_dir="./gpt2-gerchef", #The output directory - El directorio de salida
    overwrite_output_dir=True, #overwrite the content of the output directory - (sobrescribir el contenido del directorio de salida)
    num_train_epochs=3, # number of training epochs - (número de épocas de entrenamiento)
    per_device_train_batch_size=15, # batch size for training - (tamaño de lote para entrenamiento)
    per_device_eval_batch_size=30,  # batch size for evaluation - (tamaño de lote para evaluación)
    eval_steps = 400, # Number of update steps between two evaluations. - (Número de pasos de actualización entre dos evaluaciones.)
    save_steps=800, # after # steps model is saved - (después de # pasos se guarda el modelo)
    warmup_steps=500,# number of warmup steps for learning rate scheduler - (número de pasos de calentamiento para el programador de tasa de aprendizaje)
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:911: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/644M [00:00<?, ?B/s]

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 2            |        cudaMalloc retries: 2         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   10645 MB |   10669 MB |   36595 GB |   36585 GB |\n|       from large pool |   10629 MB |   10653 MB |   36591 GB |   36581 GB |\n|       from small pool |      15 MB |      15 MB |       4 GB |       4 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |   10645 MB |   10669 MB |   36595 GB |   36585 GB |\n|       from large pool |   10629 MB |   10653 MB |

#### Entrenar y guardar el modelo
Para entrenar el modelo, simplemente podemos ejecutar Trainer.train().

In [8]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 15744
  Num Epochs = 3
  Instantaneous batch size per device = 15
  Total train batch size (w. parallel, distributed & accumulation) = 15
  Gradient Accumulation steps = 1
  Total optimization steps = 3150


Step,Training Loss
500,2.935600
1000,2.344300
1500,2.070700
2000,1.953300
2500,1.829800
3000,1.778400


Saving model checkpoint to ./gpt2-gerchef/checkpoint-800
Configuration saved in ./gpt2-gerchef/checkpoint-800/config.json
Model weights saved in ./gpt2-gerchef/checkpoint-800/pytorch_model.bin
Saving model checkpoint to ./gpt2-gerchef/checkpoint-1600
Configuration saved in ./gpt2-gerchef/checkpoint-1600/config.json
Model weights saved in ./gpt2-gerchef/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to ./gpt2-gerchef/checkpoint-2400
Configuration saved in ./gpt2-gerchef/checkpoint-2400/config.json
Model weights saved in ./gpt2-gerchef/checkpoint-2400/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3150, training_loss=2.1326900615389386, metrics={'train_runtime': 4182.3019, 'train_samples_per_second': 11.293, 'train_steps_per_second': 0.753, 'total_flos': 3085336313856000.0, 'train_loss': 2.1326900615389386, 'epoch': 3.0})

####Una vez finalizado el entrenamiento, puede guardar el modelo llamando a save_model(). Esto guardará el modelo entrenado en nuestro output_dir desde nuestros TrainingArguments.

In [9]:
trainer.save_model()

Saving model checkpoint to ./gpt2-gerchef
Configuration saved in ./gpt2-gerchef/config.json
Model weights saved in ./gpt2-gerchef/pytorch_model.bin


#####Para probar el modelo, vamos a utilizar otro elemento destacado de la biblioteca de transformadores llamado tubería. Los pipelines son objetos que ofrecen una API simple dedicada a varias tareas, entre otras también la generación de texto.

In [10]:
from transformers import pipeline

#chef = pipeline('text-generation',model='./gpt2-gerchef', tokenizer='anonymous-german-nlp/german-gpt2',config={'max_length':800})
pipe = pipeline('text-generation', model='./gpt2-gerchef', tokenizer='anonymous-german-nlp/german-gpt2')
#result = chef('Zuerst Hähnchen')[0]['generated_text']


loading configuration file ./gpt2-gerchef/config.json
Model config GPT2Config {
  "_name_or_path": "./gpt2-gerchef",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "use_cache": true,
  "vocab_size": 52000
}

load

In [11]:
pipe('Die Nudeln Kochen, Fleisch anbraten')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Die Nudeln Kochen, Fleisch anbraten und zusammen mit den Paprikastreifen zum Fleisch geben. Mit Salz, Pfeffer, Paprika und Koriander würzen. Alles mit dem Olivenöl vermengen, dass das Gemüse gerade bedeckt ist. Mit Salz'}]

In [12]:
pipe('Zuerst Hähnchen')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Zuerst Hähnchenbrustfilets aufschneiden, einwickeln und auf der Hautseite mit den anderen Fischfilets kurz anbraten. Hähnchenfleisch und Zwiebeln in etwas Öl bräunen oder klein schneiden. Zitrone würfeln. Zwiebeln in Öl'}]

In [13]:
pipe('Der beste Weg, um einen Schokoladenkuchen zuzubereiten, ist')


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Der beste Weg, um einen Schokoladenkuchen zuzubereiten, ist das Backrohr, in dessen Deckel sich die Füllung befindet.Die Ränder am Teig aus dem Kühlschrank nehmen.Den Backofen vorheizen.Eine Auflaufform mit Mehl bestäuben'}]

In [18]:
!zip -r /content/gpt2-gerchef.zip /content/gpt2-gerchef

  adding: content/gpt2-gerchef/ (stored 0%)
  adding: content/gpt2-gerchef/training_args.bin (deflated 49%)
  adding: content/gpt2-gerchef/config.json (deflated 51%)
  adding: content/gpt2-gerchef/checkpoint-1600/ (stored 0%)
  adding: content/gpt2-gerchef/checkpoint-1600/rng_state.pth (deflated 27%)
  adding: content/gpt2-gerchef/checkpoint-1600/training_args.bin (deflated 49%)
  adding: content/gpt2-gerchef/checkpoint-1600/config.json (deflated 51%)
  adding: content/gpt2-gerchef/checkpoint-1600/pytorch_model.bin (deflated 10%)
  adding: content/gpt2-gerchef/checkpoint-1600/optimizer.pt (deflated 8%)
  adding: content/gpt2-gerchef/checkpoint-1600/trainer_state.json (deflated 56%)
  adding: content/gpt2-gerchef/checkpoint-1600/scheduler.pt (deflated 49%)
  adding: content/gpt2-gerchef/checkpoint-800/ (stored 0%)
  adding: content/gpt2-gerchef/checkpoint-800/rng_state.pth (deflated 27%)
  adding: content/gpt2-gerchef/checkpoint-800/training_args.bin (deflated 49%)
  adding: content/gpt